In [1]:
pip install pandas sqlalchemy


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

orders = pd.read_csv('C:\\Users\\Charita\\Downloads\\orders.csv')
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [2]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   order_id         10000 non-null  int64  
 1   user_id          10000 non-null  int64  
 2   restaurant_id    10000 non-null  int64  
 3   order_date       10000 non-null  object 
 4   total_amount     10000 non-null  float64
 5   restaurant_name  10000 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 468.9+ KB


In [4]:
users = pd.read_json("C:\\Users\\Charita\\Downloads\\users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [6]:
import sqlite3

conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

with open("C:\\Users\\Charita\\Downloads\\restaurants.sql", "r") as f:
    cursor.executescript(f.read())

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [7]:
merged_1 = orders.merge(
    users,
    on="user_id",
    how="left"
)


In [8]:
final_df = merged_1.merge(
    restaurants,
    on="restaurant_id",
    how="left"
)


In [11]:
final_df.isnull().sum()


order_id             0
user_id              0
restaurant_id        0
order_date           0
total_amount         0
restaurant_name_x    0
name                 0
city                 0
membership           0
restaurant_name_y    0
cuisine              0
rating               0
dtype: int64

In [13]:
final_df["order_date"] = pd.to_datetime(
    final_df["order_date"],
    dayfirst=True
)


In [21]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [22]:
final_df = pd.read_csv("final_food_delivery_dataset.csv")
final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,2023-02-18,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,2023-01-18,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,2023-07-15,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,2023-10-04,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,2023-12-25,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [24]:
final_df.shape



(10000, 12)

In [25]:
final_df.columns


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')

In [26]:
final_df.isnull().sum()

order_id             0
user_id              0
restaurant_id        0
order_date           0
total_amount         0
restaurant_name_x    0
name                 0
city                 0
membership           0
restaurant_name_y    0
cuisine              0
rating               0
dtype: int64

In [28]:
df.columns


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')

In [29]:
gold_df = df[df["membership"] == "Gold"]

city_revenue = (
    gold_df.groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

city_revenue


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [30]:
cuisine_aov = (
    df.groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)
cuisine_aov

cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [31]:
user_total = (
    df.groupby("user_id")["total_amount"]
    .sum()
)

users_above_1000 = user_total[user_total > 1000]

users_above_1000.count()


np.int64(2544)

In [32]:
df["rating_range"] = pd.cut(
    df["rating"],
    bins=[3.0, 3.5, 4.0, 4.5, 5.0],
    labels=["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]
)

rating_revenue = (
    df.groupby("rating_range")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

rating_revenue


C:\Users\Charita\AppData\Local\Temp\ipykernel_7468\116998896.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"]


rating_range
4.6–5.0    2197030.75
4.1–4.5    1960326.26
3.0–3.5    1881754.57
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

In [33]:
gold_df = df[df["membership"] == "Gold"]

gold_city_aov = (
    gold_df.groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

gold_city_aov


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [34]:
restaurant_count = (
    df.groupby("cuisine")["restaurant_id"]
    .nunique()
)
restaurant_count


cuisine
Chinese    120
Indian     126
Italian    126
Mexican    128
Name: restaurant_id, dtype: int64

In [35]:
cuisine_revenue = (
    df.groupby("cuisine")["total_amount"]
    .sum()
)
cuisine_revenue


cuisine
Chinese    1930504.65
Indian     1971412.58
Italian    2024203.80
Mexican    2085503.09
Name: total_amount, dtype: float64

In [36]:
cuisine_summary = pd.DataFrame({
    "restaurant_count": restaurant_count,
    "total_revenue": cuisine_revenue
}).sort_values("restaurant_count")

cuisine_summary


,restaurant_count,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [37]:
total_orders = len(df)
gold_orders = len(df[df["membership"] == "Gold"])

gold_percentage = round((gold_orders / total_orders) * 100)
gold_percentage


50

In [38]:
restaurant_stats = df.groupby("restaurant_name_y").agg(
    total_orders=("order_id", "count"),
    avg_order_value=("total_amount", "mean")
)

filtered = restaurant_stats[restaurant_stats["total_orders"] < 20]
top_restaurant = filtered.sort_values("avg_order_value", ascending=False).head(1)
top_restaurant


,total_orders,avg_order_value
restaurant_name_y,,
Restaurant_294,13,1040.222308


In [39]:
combo_revenue = (
    df.groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

combo_revenue


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [40]:
df["order_date"] = pd.to_datetime(df["order_date"])

df["quarter"] = df["order_date"].dt.quarter

quarter_revenue = (
    df.groupby("quarter")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

quarter_revenue


quarter
3    2037385.10
4    2018263.66
1    2010626.64
2    1945348.72
Name: total_amount, dtype: float64

In [41]:
gold_df = df[df["membership"] == "Gold"]

total_gold_orders = gold_df.shape[0]
total_gold_orders


4987

In [42]:
hyderabad_revenue = df[df["city"] == "Hyderabad"]["total_amount"].sum()
round(hyderabad_revenue)


1889367

In [43]:
distinct_users = df["user_id"].nunique()
distinct_users


2883

In [44]:
gold_df = df[df["membership"] == "Gold"]

gold_aov = round(gold_df["total_amount"].mean(), 2)
gold_aov


np.float64(797.15)

In [45]:
high_rating_orders = df[df["rating"] >= 4.5]["order_id"].count()
high_rating_orders


np.int64(3374)

In [46]:
top_gold_city = gold_city_aov.index[0]  # top city from previous result
orders_top_gold_city = gold_df[gold_df["city"] == top_gold_city]["order_id"].count()
orders_top_gold_city


np.int64(1337)